# Markdown 문서 분할하기

이 노트북에서는 **RecursiveCharacterTextSplitter.from_language()**로 Markdown 문서를 구조에 맞게 분할하는 방법을 알아봅니다.

## Markdown 분할 특징

Markdown은 헤더(`#`, `##`) 기반 구조를 가지므로, 이를 기준으로 분할하면 의미 있는 섹션 단위로 나눌 수 있습니다.

**Markdown 구분자 순서:**
1. `\n# ` (H1 헤더)
2. `\n## ` (H2 헤더)
3. `\n### ` (H3 헤더)
4. `\n#### ` (H4 헤더)
5. `\n- ` (리스트)
6. ` ``` ` (코드 블록)
7. `\n\n`, `\n`, ` `

---

# 1. 패키지 설치

In [ ]:
!pip install -q langchain langchain-text-splitters

# 2. Markdown 문서 분할

**코드 설명:**

```python
md_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.MARKDOWN,
    chunk_size=60,
    chunk_overlap=0
)

# metadata 추가 가능
md_docs = md_splitter.create_documents(
    [markdown_text],
    [{'source': 'https://...'}]  # 각 문서의 메타데이터
)
```

In [ ]:
from langchain_text_splitters import Language, RecursiveCharacterTextSplitter

# 샘플 Markdown 텍스트
markdown_text = ''' 
# 🦜🔗 LangChain ⚡ Building applications with LLMs through composability ⚡ 

## Quick Install
```bash
pip install langchain
```

As an open source project in a rapidly developing field, we are extremely open
    to contributions.
'''

print("=== 원본 Markdown ===")
print(markdown_text)

In [ ]:
# Markdown 전용 Splitter 생성
md_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.MARKDOWN,
    chunk_size=60,
    chunk_overlap=0
)

# 메타데이터와 함께 문서 생성
md_docs = md_splitter.create_documents(
    [markdown_text],
    [{'source': 'https://www.langchain.com'}]  # 출처 메타데이터
)

print(f"\n분할된 청크 수: {len(md_docs)}")

# 3. 분할 결과 확인

In [ ]:
# 각 청크 확인
for i, doc in enumerate(md_docs):
    print(f"\n=== 청크 {i+1} ===")
    print(f"metadata: {doc.metadata}")
    print(f"content ({len(doc.page_content)}자):")
    print(doc.page_content)

# 4. 헤더 기반 분할 (MarkdownHeaderTextSplitter)

헤더를 메타데이터로 보존하면서 분할하는 방법입니다.

In [ ]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

# 분할할 헤더 레벨 정의
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

md_header_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on
)

# 더 구조화된 Markdown 예시
structured_md = '''# LangChain 소개

LangChain은 LLM 애플리케이션 프레임워크입니다.

## 설치 방법

pip install langchain 명령어로 설치합니다.

## 주요 기능

### 체인 구성

LCEL로 체인을 구성합니다.

### RAG

검색 증강 생성을 지원합니다.
'''

header_splits = md_header_splitter.split_text(structured_md)

print("=== 헤더 기반 분할 결과 ===")
for i, doc in enumerate(header_splits):
    print(f"\n--- 청크 {i+1} ---")
    print(f"metadata: {doc.metadata}")
    print(f"content: {doc.page_content}")

---

## 두 Splitter 비교

| Splitter | 특징 |
|----------|------|
| `RecursiveCharacterTextSplitter.from_language(MARKDOWN)` | chunk_size 기반 분할 |
| `MarkdownHeaderTextSplitter` | 헤더 기반 분할, 헤더를 메타데이터로 보존 |

## 조합 사용 (권장)

```python
# 1. 헤더로 먼저 분할
header_splits = md_header_splitter.split_text(markdown)

# 2. 큰 섹션은 추가로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500)
final_splits = text_splitter.split_documents(header_splits)
```